# `cobrapy`: metabolic modeling in python

<img src='figs/hero.png' width='400', style='display: block; margin: 0 auto;'>

## Introduction to Flux Balance Analysis

#### Metabolic networks convert substrates into products and energy 

![](figs/escher.png)

Flux Balance analysis assumes a pseudo-steady-state for each metabolite in the model
<img src='figs/metabolic_map_cycle_subset.svg' width='300', style='display: block; margin: 0 auto;'>

$$
\begin{aligned}
r_2: &\rightarrow m_2\\
r_4: m_1 &\rightarrow m_2\\
r_6: m_2 &\rightarrow m_5\\
\end{aligned}
$$

We can then construct a *flux balance*
$$
v_2 + v_4 = v_6
$$

Fluxes are therefore constrained by the mass balance equations

$$
\begin{aligned}
\mathbf{S}\mathbf{v} &= 0\\
\mathbf{v}_{lb} &\le \mathbf{v} \le \mathbf{v}_{ub} \\
\end{aligned}
$$

<img src='figs/s_mat.svg' width='300', style='display: block; margin: 0 auto;'>

### History of the `cobrapy` package

Originally published in 2013:

* Ebrahim A, Lerman JA, Palsson BØ, Hyduke DR. COBRApy: COnstraints-Based Reconstruction and Analysis for Python. *BMC Syst Biol.* 2013;7(1):74. [doi.org/10.1186/1752-0509-7-74.](https://doi.org/10.1186/1752-0509-7-74)

Now taken over by the Novo Nordisk Center for Biosustainability and other open-source contributors

## Installation

1. *(optional)* First, create a new conda environment:
```
conda create --name presentation python=3.6
source activate presentation
```

2. Next, install `cobra` using pip:
```
pip install cobra
```

3. Alternatively, install a bleeding-edge version from the github repository
```
git clone git@github.com:opencobra/cobrapy.git
pip install cython
pip install -e cobrapy
```

## Constructing a model

In [1]:
import cobra
print(cobra.__version__)

0.8.2


### Metabolites

Metabolites represent an individual species in the model. For models with multiple compartments, an individual metabolite has to be created for each compartment.

In [2]:
glc = cobra.Metabolite(id='glc', name='D-Glucose', formula='C6H12O6', charge=None)

Chemical formulas are parsed automatically

In [3]:
glc.elements

{'C': 6, 'H': 12, 'O': 6}

In [4]:
glc.formula_weight

180.15588

### Reactions
Reactions are groupings of metabolites with a defined stoichiometry. 

In [5]:
# Create some other needed metabolites
pyr = cobra.Metabolite(id='pyr', name='Pyruvate', formula='C3H3O3')
pep = cobra.Metabolite(id='pep', name='Phosphoenolpyruvate', formula='C3H2O6P')
g6p = cobra.Metabolite(id='g6p', name='D-Glucose 6-phosphate', formula='C6H11O9P')

GLCpts = cobra.Reaction(id='GLCpts', name='D-glucose transport via PEP:Pyr PTS',
                        lower_bound=0., upper_bound=1000.)

GLCpts.add_metabolites({
 glc: -1.0,
 pep: -1.0,
 g6p: 1.0,
 pyr: 1.0,
})

In [6]:
GLCpts.reaction

'glc + pep --> g6p + pyr'

In [7]:
GLCpts.check_mass_balance()

{}

In [8]:
# Remove pyruvate
GLCpts.add_metabolites({pyr: -1})
GLCpts.reaction

'glc + pep --> g6p'

In [9]:
GLCpts.check_mass_balance()

{'C': -3.0, 'H': -3.0, 'O': -3.0}

In [10]:
GLCpts.add_metabolites({pyr: 1})

#### Reaction bounds

Reactions have both a `lower_bound` and `upper_bound` on the flux which can pass through them. These can be manipulated through object attributes:

In [11]:
GLCpts.lower_bound = -10
GLCpts.lower_bound

-10

The `reversibility` attribute determines whether both positive an negative flux is supported

In [12]:
GLCpts.reversibility

True

Bounds can be set at once through the `bounds` attribute, or zeroed using the `knock_out` method

In [13]:
GLCpts.knock_out()
GLCpts.bounds

(0, 0)

In [14]:
GLCpts.bounds = (0, 1000)
GLCpts.upper_bound

1000

In [15]:
GLCpts.reversibility

False

### Models
Models form the basis of most of the analysis in `cobrapy`, and are comprised of sets of metabolites and reactions

In [16]:
# Create some extra metabolites and reactions
atp = cobra.Metabolite('atp')
adp = cobra.Metabolite('adp')
co2 = cobra.Metabolite('co2')

upper_glycol = cobra.Reaction(id='upg')
lower_glycol = cobra.Reaction(id='lwg')
atp_maint    = cobra.Reaction(id='atpm')
pyr_kin      = cobra.Reaction(id='pyk')


upper_glycol.add_metabolites({
    g6p: -1,
    pyr: 1,
    pep: 1,    
})

lower_glycol.add_metabolites({
    pyr: -1,
    adp: -3,
    co2: 3,
    atp: 3,
})

atp_maint.add_metabolites({
    atp: -1,
    adp: 1,
})

pyr_kin.add_metabolites({
    pyr: -1,
    adp: -1,
    atp: 1,
    pep: 1,
})

Create the model object

In [17]:
test_model = cobra.Model('test')

Add reactions to the model. Note that metabolites are added automatically

In [18]:
test_model.add_reactions([upper_glycol, lower_glycol, atp_maint, GLCpts])

# Add exchanges for the boundary species
co2_bound = test_model.add_boundary(test_model.metabolites.co2)
glc_bound = test_model.add_boundary(test_model.metabolites.glc)
glc_bound.lower_bound = -10

test_model.metabolites

[<Metabolite g6p at 0x118ca5860>,
 <Metabolite pyr at 0x118ca57b8>,
 <Metabolite pep at 0x118ca57f0>,
 <Metabolite adp at 0x118caf7b8>,
 <Metabolite co2 at 0x118caf828>,
 <Metabolite atp at 0x118caf7f0>,
 <Metabolite glc at 0x118c90978>]

In [19]:
test_model.reactions

[<Reaction upg at 0x118caf860>,
 <Reaction lwg at 0x118caf8d0>,
 <Reaction atpm at 0x118caf940>,
 <Reaction GLCpts at 0x118ca5898>,
 <Reaction EX_co2 at 0x118ca5be0>,
 <Reaction EX_glc at 0x118ca5da0>]

Models link metabolites and reactions: lower-level objects are aware of their links to other objects in the model

In [20]:
test_model.metabolites.pyr.reactions

frozenset({<Reaction GLCpts at 0x118ca5898>,
           <Reaction lwg at 0x118caf8d0>,
           <Reaction upg at 0x118caf860>,
           <Reaction pyk at 0x118caf978>})

In [21]:
test_model.reactions.upg.reactants

[<Metabolite g6p at 0x118ca5860>]

### Copying models
Models can be deep-copied to allow for divergent edits to metabolite and reaction connectivity

In [22]:
copied_model = test_model.copy()
copied_model.reactions.upg.remove_from_model()
copied_model.reactions.GLCpts.remove_from_model()

Note the different memory locations of the reaction objects

In [23]:
copied_model.reactions

[<Reaction lwg at 0x118caf278>,
 <Reaction atpm at 0x118caf400>,
 <Reaction EX_co2 at 0x118caf518>,
 <Reaction EX_glc at 0x118caf630>]

In [24]:
test_model.reactions

[<Reaction upg at 0x118caf860>,
 <Reaction lwg at 0x118caf8d0>,
 <Reaction atpm at 0x118caf940>,
 <Reaction GLCpts at 0x118ca5898>,
 <Reaction EX_co2 at 0x118ca5be0>,
 <Reaction EX_glc at 0x118ca5da0>]

### Reading and writing `cobra` models

`cobrapy` can read and write models to SBML, the cobra MATLAB format, and also to a standalone `.json` / `yaml` format.

#### SBML
Example taken from http://systemsbiology.ucsd.edu/InSilicoOrganisms/Ecoli/EcoliSBML

In [25]:
ecoli = cobra.io.read_sbml_model('data/Ec_core_flux1.xml')
ecoli

Name,Ec_core
Memory address,0x0118dae320
Number of metabolites,63
Number of reactions,77
Objective expression,-1.0*Biomass_Ecoli_core_N__w_GAM__reverse_d9451 + 1.0*Biomass_Ecoli_core_N__w_GAM_
Compartments,"Extra_organism, Cytosol"


In [26]:
ecoli.metabolites.atp_c.reactions

frozenset({<Reaction Biomass_Ecoli_core_N__w_GAM_ at 0x118dccc50>,
           <Reaction ATPS4r at 0x118dccc88>,
           <Reaction PPCK at 0x118dfccf8>,
           <Reaction SUCOAS at 0x118e0f128>,
           <Reaction PYK at 0x118e05550>,
           <Reaction PGK at 0x118dfca58>,
           <Reaction PPS at 0x118e05278>,
           <Reaction ADK1 at 0x118dcc6a0>,
           <Reaction PFK at 0x118df0b38>,
           <Reaction ACKr at 0x118dcc358>,
           <Reaction ATPM at 0x118dccba8>})

#### JSON

In [27]:
cobra.io.save_json_model(test_model, 'data/test_model.json', pretty=True)

with open('data/test_model.json') as myfile:
    head = [next(myfile) for x in range(15)]
print(''.join(head))
print('...')

{
    "compartments": {},
    "genes": [],
    "id": "test",
    "metabolites": [
        {
            "compartment": "",
            "id": "adp",
            "name": ""
        },
        {
            "compartment": "",
            "id": "atp",
            "name": ""
        },

...


#### YAML

In [28]:
cobra.io.save_yaml_model(test_model, 'data/test_model.yaml')

with open('data/test_model.yaml') as myfile:
    head = [next(myfile) for x in range(15)]
print(''.join(head))
print('...')

!!omap
- reactions:
  - !!omap
    - id: EX_co2
    - name: ' exchange'
    - metabolites: !!omap
      - co2: -1
    - lower_bound: -1000.0
    - upper_bound: 1000.0
    - gene_reaction_rule: ''
  - !!omap
    - id: EX_glc
    - name: D-Glucose exchange
    - metabolites: !!omap
      - glc: -1

...
